In [52]:
import pandas as pd
import numpy as np
import plotly as plt
import plotly.express as px
from datetime import datetime
from scipy.fftpack import fft, ifft, fftfreq

First, we create a dataframe of Building 9's Steam, Chilled Water, and Power Usage. We convert the dates into a usable date format, convert the different values to common units (kBTUs), and combine the data from multiple transformers into one power column. The resulting dataframe is displayed.

In [36]:
# % Conversions
# % 1kwh = 3412.14 BTU
# % 1 THR = 12000 BTU
# % 1 lbs steam = 1100 BTU

# % Square feet
# % Building 9: 77,414  
# % Building 76: 367,689
# % Building E60: 30,130


# Convert to BTU %
PowertokBTU = 3.41214
CWtokBTU = 12.000
SteamtokBTU = 1.100

# % Building Area %
Bldg9SF = 77414
Bldg76SF = 367689

# % Process Loads %
Bldg9SteamProcess = 6
Bldg9CWProcess = 5
Bldg76SteamProcess = 9
Bldg76CWProcess = 4

# Import Data and convert to common units
data9 = pd.read_csv('Building9.csv')
data9 = data9.rename(columns={'Unnamed: 0': 'Time', 'M9STMFLOW': 'Steam', 'M9CHWTONS': "Chilled Water" })
data9["Time"] = data9["Time"].map(lambda str: datetime.strptime(str, "%m/%d/%Y %H:%M" )) ##convert dates to datetime object
data9["Steam"] = data9["Steam"]*SteamtokBTU
data9["Chilled Water"] = data9["Chilled Water"]*CWtokBTU
data9["Power"] = (data9["M9TFRARealPower"] + data9["M9TFRBRealPower"])*PowertokBTU ##combine the two transformers
data9 = data9.drop(labels=["M9TFRARealPower", "M9TFRBRealPower"], axis=1)
data9

,Time,Steam,Chilled Water,TMP C,TMP F,Power
0,2019-01-01 00:00:00,2313.840064,355.632820,5.6,42.08,119.085230
1,2019-01-01 01:00:00,2085.009583,379.660080,3.9,39.02,118.594116
2,2019-01-01 02:00:00,1717.793140,477.202606,3.3,37.94,123.034917
3,2019-01-01 03:00:00,1706.638184,808.289520,2.8,37.04,120.815738
4,2019-01-01 04:00:00,1472.311999,751.712219,3.3,37.94,121.889061
...,...,...,...,...,...,...
8731,2019-12-30 20:00:00,2035.916846,388.572968,4.0,39.20,137.381312
8732,2019-12-30 21:00:00,1872.775782,375.457420,3.3,37.94,140.519424
8733,2019-12-30 22:00:00,1934.195447,370.756416,4.0,39.20,135.654000
8734,2019-12-30 23:00:00,1980.725098,363.329178,3.9,39.02,132.359963


Now, we create a graph of the Steam usage, over time, with each measurement taken hourly. We've also included important statistics for  Notice that there are clearly erroneous values (i.e. negative values). 

In [53]:
max_steam = round(max(data9["Steam"])) ##data taken hourly
min_steam = round(min(data9["Steam"]))
avg_steam = round(data9["Steam"].mean())
std_steam = round(data9["Steam"].std())

fig = px.scatter(x=data9["Time"], y=data9['Steam'], title="Building 9 Steam (hourly)", 
                labels={"x":"Time", "y":"Total kBTU"}) ##removing negative values

fig.add_annotation(text='Max: '+ str(max_steam),
                  xref="paper", yref="paper",
                  x=0, y=0, showarrow=False)
fig.add_annotation(text='Min: '+ str(min_steam),
                  xref="paper", yref="paper",
                  x=0, y=0.05, showarrow=False)
fig.add_annotation(text='Mean: '+ str(avg_steam),
                  xref="paper", yref="paper",
                  x=0, y=0.15, showarrow=False)
fig.add_annotation(text='Standard Deviation: '+ str(std_steam),
                  xref="paper", yref="paper",
                  x=0, y=0.1, showarrow=False)


fig.show()

We now do the same thing, but this time just removing negative values that are likely just a bad sensor reading. 

In [38]:
bad_indices = []
for i in range(len(data9["Steam"])):
    if data9["Steam"][i] < 0:
        bad_indices.append(i)
data9_clean = data9.drop(bad_indices)
steam_cleaned = data9_clean["Steam"]

max_steam_clean = round(max(steam_cleaned)) ##data taken hourly
min_steam_clean = round(min(steam_cleaned))
avg_steam_clean = round(steam_cleaned.mean())
std_steam_clean = round(steam_cleaned.std())

fig = px.scatter(x=data9_clean["Time"], y=steam_cleaned, title="Building 9 Steam (hourly)", 
                labels={"x":"Time", "y":"Total kBTU"}) ##removing negative values

fig.add_annotation(text='Max: '+ str(max_steam_clean),
                  xref="paper", yref="paper",
                  x=0, y=0, showarrow=False)
fig.add_annotation(text='Min: '+ str(min_steam_clean),
                  xref="paper", yref="paper",
                  x=0, y=0.05, showarrow=False)
fig.add_annotation(text='Mean: '+ str(avg_steam_clean),
                  xref="paper", yref="paper",
                  x=0, y=0.15, showarrow=False)
fig.add_annotation(text='Standard Deviation: '+ str(std_steam_clean),
                  xref="paper", yref="paper",
                  x=0, y=0.1, showarrow=False)


fig.show()

In [42]:
freq = {}
for i in range(round(max_steam), 0, -1):
    for measurement in data9_clean["Steam"]:
        if measurement > i:
            if i not in freq:
                freq[i] = 1
            else:
                freq[i] += 1
total = len(data9_clean["Steam"])
freq_array = []
for i in freq.values():
    freq_array.append(i/total*100)

fig = px.scatter(x=freq_array, y=freq.keys(), title="Duration Curve for Building 9 Steam", labels={"x":"Percent of Year", "y":"Total kBTU"})
fig.show()

In [50]:
max_chw = round(max(data9["Chilled Water"])) ##data taken hourly
min_chw = round(min(data9["Chilled Water"]))
avg_chw = round(data9["Chilled Water"].mean())
std_chw = round(data9["Chilled Water"].std())

fig = px.scatter(x=data9["Time"], y=data9['Chilled Water'], title="Building 9 Chilled Water (hourly)", 
                labels={"x":"Time", "y":"Total BTU"}) 

fig.add_annotation(text='Max: '+ str(max_chw),
                  xref="paper", yref="paper",
                  x=0, y=0.9, showarrow=False)
fig.add_annotation(text='Min: '+ str(min_chw),
                  xref="paper", yref="paper",
                  x=0, y=0.85, showarrow=False)
fig.add_annotation(text='Mean: '+ str(avg_chw),
                  xref="paper", yref="paper",
                  x=0, y=0.8, showarrow=False)
fig.add_annotation(text='Standard Deviation: '+ str(std_chw),
                  xref="paper", yref="paper",
                  x=0, y=0.75, showarrow=False)


fig.show()

In [51]:
freq = {}
for i in range(round(max_chw), 0, -1):
    for measurement in data9["Chilled Water"]:
        if measurement > i:
            if i not in freq:
                freq[i] = 1
            else:
                freq[i] += 1
total = len(data9["Chilled Water"])
freq_array = []
for i in freq.values():
    freq_array.append(i/total*100)

fig = px.scatter(x=freq_array, y=freq.keys(), title="Duration Curve for Building 9 Chilled Water", labels={"x":"Percent of Year", "y":"Total kBTU"})
fig.show()

In [49]:
max_power = round(max(data9["Power"])) ##data taken hourly
min_power = round(min(data9["Power"]))
avg_power = round(data9["Power"].mean())
std_power = round(data9["Power"].std())

fig = px.scatter(x=data9["Time"], y=data9['Power'], title="Building 9 Power (Hourly)", 
                labels={"x":"Time", "y":"Total BTU"})

fig.add_annotation(text='Max: '+ str(max_power),
                  xref="paper", yref="paper",
                  x=0, y=0.9, showarrow=False)
fig.add_annotation(text='Min: '+ str(min_power),
                  xref="paper", yref="paper",
                  x=0, y=0.85, showarrow=False)
fig.add_annotation(text='Mean: '+ str(avg_power),
                  xref="paper", yref="paper",
                  x=0, y=0.8, showarrow=False)
fig.add_annotation(text='Standard Deviation: '+ str(std_power),
                  xref="paper", yref="paper",
                  x=0, y=0.75, showarrow=False)


fig.show()